<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Feature-engineering" data-toc-modified-id="Feature-engineering-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Feature engineering</a></span></li><li><span><a href="#Модели" data-toc-modified-id="Модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Модели</a></span><ul class="toc-item"><li><span><a href="#Ridge" data-toc-modified-id="Ridge-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Ridge</a></span></li><li><span><a href="#RandomForestRegressor" data-toc-modified-id="RandomForestRegressor-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>RandomForestRegressor</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>CatBoost</a></span><ul class="toc-item"><li><span><a href="#CatBoost-с-Кросс-Валидацией" data-toc-modified-id="CatBoost-с-Кросс-Валидацией-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>CatBoost с Кросс-Валидацией</a></span></li></ul></li><li><span><a href="#XGBRegressor" data-toc-modified-id="XGBRegressor-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>XGBRegressor</a></span></li><li><span><a href="#ETNA" data-toc-modified-id="ETNA-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>ETNA</a></span><ul class="toc-item"><li><span><a href="#Без-экзогенных-данных" data-toc-modified-id="Без-экзогенных-данных-3.5.1"><span class="toc-item-num">3.5.1&nbsp;&nbsp;</span>Без экзогенных данных</a></span></li><li><span><a href="#C-экзогенными-данными" data-toc-modified-id="C-экзогенными-данными-3.5.2"><span class="toc-item-num">3.5.2&nbsp;&nbsp;</span>C экзогенными данными</a></span></li></ul></li></ul></li><li><span><a href="#Общий-вывод:" data-toc-modified-id="Общий-вывод:-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод:</a></span></li></ul></div>

## Загрузка данных

In [ ]:
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="etna.settings")


import json
import time
import folium
import requests
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
from plotly.subplots import make_subplots
from matplotlib.ticker import FuncFormatter
from datetime import datetime, timedelta
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score 
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Выведем все поля
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

#Загружаем основной датасет
yandex_disk_url = "https://disk.yandex.ru/d/M1Yk0D6IfHis2A"
response = requests.get("https://cloud-api.yandex.net/v1/disk/public/resources/download",
                        params={"public_key": yandex_disk_url})

download_url = response.json()["href"]

main_df = pd.read_csv(download_url
                      , parse_dates=['timestamp']
                      , sep=',')

main_df.head()

In [ ]:
# Население районов за 2024год, обновляем данные
# Источник https://77.rosstat.gov.ru/folder/64634 

population_2024 = {
    'Академический район': 114932,
    'Алексеевский район': 79377,
    'Алтуфьевский район': 56008,
    'Бабушкинский район': 90087,
    'Басманный район': 112320,
    'Бескудниковский район': 81647,
    'Бутырский район': 72177,
    'Войковский район': 72588,
    'Гагаринский район': 80211,
    'Головинский район': 109389,
    'Даниловский район': 104718,
    'Дмитровский район': 104847,
    'Донской район': 51202,
    'Краснопахорский район': 36136,
    'Красносельский район': 46340,
    'Ломоносовский район': 86867,
    'Лосиноостровский район': 83779,
    'Мещанский район': 55669,
    'Можайский район': 146666,
    'Молжаниновский район': 14907,
    'Нагорный район': 76015,
    'Нижегородский район': 52114,
    'Ново-Переделкино': 121741,
    'Обручевский район': 91308,
    'Орехово-Борисово Южное': 146515,
    'Останкинский район': 65211,
    'Пресненский район': 129041,
    'Рязанский район': 109884,
    'Савёловский район': 61147,
    'Таганский район': 134535,
    'Тверской район': 76889,
    'Тимирязевский район': 84168,
    'Тропарёво-Никулино': 129268,
    'Филимонковский район': 137565,
    'Хорошёвский район': 76407,
    'Южнопортовый район': 73321,
    'Ярославский район': 99398,
    'район Арбат': 36942,
    'район Аэропорт': 82804,
    'район Беговой': 43662,
    'район Бекасово': 35814,
    'район Бибирево': 159377,
    'район Бирюлёво Восточное': 155760,
    'район Бирюлёво Западное': 84910,
    'район Богородское': 108907,
    'район Братеево': 100282,
    'район Вешняки': 121088,
    'район Внуково': 128383,
    'район Вороново': 27777,
    'район Восточное Дегунино': 101126,
    'район Восточное Измайлово': 74200,
    'район Восточный': 12611,
    'район Выхино-Жулебино': 242376,
    'район Гольяново': 164305,
    'район Дорогомилово': 76571,
    'район Замоскворечье': 56230,
    'район Западное Дегунино': 85448,
    'район Зюзино': 124371,
    'район Зябликово': 133650,
    'район Ивановское': 125211,
    'район Измайлово': 110331,
    'район Капотня': 32128,
    'район Коммунарка': 180477,
    'район Коньково': 154105,
    'район Коптево': 104775,
    'район Косино-Ухтомский': 90207,
    'район Котловка': 66304,
    'район Крылатское': 83465,
    'район Крюково': 115081,
    'район Кузьминки': 140738,
    'район Кунцево': 152171,
    'район Куркино': 33676,
    'район Левобережный': 56573,
    'район Лефортово': 113163,
    'район Лианозово': 83759,
    'район Люблино': 185286,
    'район Марфино': 37549,
    'район Марьина Роща': 62551,
    'район Марьино': 273463,
    'район Матушкино': 40220,
    'район Метрогородок': 40798,
    'район Митино': 198911,
    'район Москворечье-Сабурово': 87843,
    'район Нагатино-Садовники': 83794,
    'район Нагатинский Затон': 121411,
    'район Некрасовка': 115849,
    'район Новогиреево': 99906,
    'район Новокосино': 108479,
    'район Орехово-Борисово Северное': 124071,
    'район Отрадное': 188431,
    'район Очаково-Матвеевское': 127231,
    'район Перово': 133481,
    'район Печатники': 90529,
    'район Покровское-Стрешнево': 68688,
    'район Преображенское': 89475,
    'район Проспект Вернадского': 65913,
    'район Раменки': 154283,
    'район Ростокино': 52503,
    'район Савёлки': 37862,
    'район Свиблово': 60437,
    'район Северное Бутово': 97376,
    'район Северное Измайлово': 85448,
    'район Северное Медведково': 130718,
    'район Северное Тушино': 172664,
    'район Северный': 56537,
    'район Силино': 41971,
    'район Сокол': 58217,
    'район Соколиная Гора': 93960,
    'район Сокольники': 60852,
    'район Солнцево': 137612,
    'район Старое Крюково': 41028,
    'район Строгино': 162885,
    'район Текстильщики': 102030,
    'район Троицк': 127378,
    'район Тёплый Стан': 133400,
    'район Фили-Давыдково': 112759,
    'район Филёвский Парк': 112403,
    'район Хамовники': 108015,
    'район Ховрино': 102047,
    'район Хорошёво-Мнёвники': 183677,
    'район Царицыно': 117967,
    'район Чертаново Северное': 109627,
    'район Чертаново Центральное': 111581,
    'район Чертаново Южное': 156178,
    'район Черёмушки': 108619,
    'район Щербинка': 136323,
    'район Щукино': 112002,
    'район Южное Бутово': 182030,
    'район Южное Медведково': 88821,
    'район Южное Тушино': 113999,
    'район Якиманка': 27416,
    'район Ясенево': 176629
}

In [ ]:
# Заменяю значения населения районов на более точные из росстата
for district in population_2024.keys():
    main_df.loc[main_df.district_name == district, 'population_district_2023'] = population_2024.get(district)

## Feature engineering

In [ ]:
# Вытащим все возможные временные данные
main_df['month'] = main_df['timestamp'].dt.month
main_df['day_name'] = main_df['timestamp'].dt.day_name()
main_df['hour'] = main_df['timestamp'].dt.hour
main_df['week_number'] = main_df['timestamp'].dt.isocalendar().week

In [ ]:
# Список районов за МКАД (Новая Москва, Зеленоград, и пр.)
outside_mkad = [
    'Краснопахорский район', 'Филимонковский район', 'район Внуково',
    'район Вороново', 'район Бекасово', 'район Коммунарка',
    'район Крюково', 'район Матушкино', 'район Савёлки',
    'район Силино', 'район Старое Крюково', 'район Троицк',
    'район Щербинка', 'район Новокосино', 'район Косино-Ухтомский', 'район Некрасовка', 'район Восточный', 'район Северное Бутово',
    'район Солнцево', 'район Ново-Переделкино', 'район Северный', 'Молжаниновский район', 'район Куркино', 'район Митино'
]

# Создаём булевый признак: True — внутри МКАД, False — за МКАД
main_df['inside_mkad'] = ~main_df['district_name'].isin(outside_mkad)


inside_sadovoe = [
    'район Арбат', 'Тверской район', 'Мещанский район',
    'район Якиманка', 'район Замоскворечье', 'Басманный район',
    'Пресненский район', 'Таганский район', 'район Хамовники', 'Красносельский район'
]

# Булевый признак: находится ли район внутри Садового кольца
main_df['inside_sadovoe'] = main_df['district_name'].isin(inside_sadovoe)


inside_ttk = inside_sadovoe + [
    'район Дорогомилово', 'район Сокольники', 'район Сокол',
    'район Аэропорт', 'район Беговой', 'район Марьина Роща', 'район Лефортово', 'Донской район'
]

# Булевый признак: находится ли район внутри ТТК
main_df['inside_ttk'] = main_df['district_name'].isin(inside_ttk)

In [ ]:
# Координаты центра Москвы 
center_lat, center_lon = 55.7558, 37.6176

main_df['distance_to_center_km'] = 6371 * 2 * np.arcsin(
    np.sqrt(
        np.sin(np.radians((main_df['lat'] - center_lat) / 2))**2 +
        np.cos(np.radians(center_lat)) * np.cos(np.radians(main_df['lat'])) *
        np.sin(np.radians((main_df['lon'] - center_lon) / 2))**2
    )
)

In [ ]:
# Относительная стоимость квадратного метра
main_df['price_relative'] = main_df['price_m2'] / main_df.groupby('administrative_region')['price_m2'].transform('mean')

# Относительная загруженность метро
main_df['station_load_relative'] = main_df['station_load_norm'] / main_df.groupby('administrative_region')['station_load_norm'].transform('mean')

In [ ]:
# Временной лаг такси 3ч, 6ч, 24ч, 7дней
main_df['n_taxi_start_lag3'] = main_df.groupby('district_name')['n_taxi_start'].shift(3)
main_df['n_taxi_start_lag6'] = main_df.groupby('district_name')['n_taxi_start'].shift(6)
main_df['n_taxi_start_lag24'] = main_df.groupby('district_name')['n_taxi_start'].shift(24)
main_df['n_taxi_start_lag7day'] = main_df.groupby('district_name')['n_taxi_start'].shift(168)

# Временной лаг каршеринга
main_df['n_carsharing_start_lag24'] = main_df.groupby('district_name')['n_carsharing_start'].shift(24)
main_df['n_carsharing_start_lag168'] = main_df.groupby('district_name')['n_carsharing_start'].shift(168)

In [ ]:
# Скользящее среднее за 3ч, 6ч, 24ч, 7дней
main_df['n_taxi_start_rolling3_mean'] = (
    main_df.groupby('district_name')['n_taxi_start']
    .transform(lambda x: x.shift(1).rolling(3, min_periods=1)
    .mean()))
main_df['n_taxi_start_rolling6_mean'] = (
    main_df.groupby('district_name')['n_taxi_start']
    .transform(lambda x: x.shift(1).rolling(6, min_periods=1)
    .mean()))
main_df['n_taxi_start_rolling24_mean'] = (
    main_df.groupby('district_name')['n_taxi_start']
    .transform(lambda x: x.shift(1).rolling(24, min_periods=1)
    .mean()))
main_df['n_taxi_start_rolling7day_mean'] = (
    main_df.groupby('district_name')['n_taxi_start']
    .transform(lambda x: x.shift(1).rolling(168, min_periods=1)
    .mean()))

In [ ]:
#Погодные фичи
main_df['is_heatwave'] = (main_df['temperature'] > 30).astype(int)
main_df['is_freezing'] = (main_df['temperature'] < -5).astype(int)
main_df['is_high_wind'] = (main_df['wind_speed'] > 10).astype(int)
main_df['is_high_humidity'] = (main_df['humidity'] > 80).astype(int)
main_df['is_good_weather'] = main_df['weather_code'].isin([1, 2]).astype(int)
main_df['is_bad_weather'] = main_df['weather_code'].isin(range(7, 26)).astype(int)

extreme_codes = [9, 11, 13, 16, 18, 20, 22, 23, 24, 25]
main_df['is_extreme_weather'] = main_df['weather_code'].isin(extreme_codes).astype(int)

Добавим еще пару признаков для отслеживания динамики, в первую очередь добавляем первую производную, если результат меньше 0, происходит падение спроса, больше 0 соотвественно рост.

Также добавляем разность за 2 часа, для отслуживания резких сдвигов.

И самое главное добавляю разность между этими двумя показтелями, который будет отвечать за ускорение > 0 — тренд роста ускоряется, < 0 — тренд роста замедляется(или падение ускоряется), = 0 — стабильный тренд.

In [ ]:
main_df['taxi_diff_1h'] = main_df.groupby('district_name')['n_taxi_start'].shift(1).diff(1)
main_df['taxi_diff_2h'] = main_df.groupby('district_name')['n_taxi_start'].shift(1).diff(2)
main_df['taxi_acceleration'] = main_df['taxi_diff_1h'] - main_df['taxi_diff_2h']

In [ ]:
main_df.isna().sum()

In [ ]:
main_df.dropna(inplace=True)

In [ ]:
main_df.isna().sum()

## Модели

### Ridge

In [ ]:
# Разбиваем датасет
train = main_df[(main_df['timestamp']>='2024-01-08 00:00:00')&(main_df['timestamp']<='2024-09-30 23:00:00')]
test =  main_df[(main_df['timestamp']>='2024-11-01 00:00:00')&(main_df['timestamp']<='2024-12-31 23:00:00')]
valid = main_df[(main_df['timestamp']>='2024-10-01 00:00:00')&(main_df['timestamp']<='2024-10-31 23:00:00')]

In [ ]:
cols_to_drop = ["quarter", 
                "district_name", 
                "n_carsharing_start", 
                "timestamp",
                "n_carsharing_end", 
                "lat", 
                "lon", 
                "n_taxi_end", 
                "administrative_region",  
                "n_taxi_start", 
                'n_taxi_start_lag3', 
                'n_taxi_start_lag6',
                'n_taxi_start_lag24',
                'n_taxi_start_lag7day',
                'n_taxi_start_rolling3_mean',
                'n_taxi_start_rolling6_mean',
                'n_taxi_start_rolling24_mean', 
                'n_taxi_start_rolling7day_mean', 
                'taxi_diff_1h', 
                'taxi_diff_2h',
                'n_carsharing_start_lag24',
                'n_carsharing_start_lag168',
                'station_load'
                ]

In [ ]:
X_train = train.drop(columns = cols_to_drop)
y_train = train['n_taxi_start']

X_test = test.drop(columns = cols_to_drop)
y_test = test['n_taxi_start']

X_valid = valid.drop(columns = cols_to_drop)
y_valid = valid['n_taxi_start']

In [ ]:
num_cols = ["temperature", "precipitation", "humidity", "wind_speed",
            'price_relative', 'distance_to_center_km',
            'taxi_acceleration', 'station_load_relative', "price_m2", "population_district_2023",
            "number_of_places", "population_near_metro"]
ohe_cols = ["season", "day_name"]
col_trans = ColumnTransformer([
  (('ohe'), OrdinalEncoder(),ohe_cols),
  ("scaler", StandardScaler(), num_cols)
], remainder='passthrough')

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [ ]:
def train_model(model):
    # Оборачиваем модель в pipeline
    pipeline = Pipeline([
        ("col_trans", col_trans),
        (model.__class__.__name__, model)
    ])
    
    # Обучение
    pipeline.fit(X_train, y_train)
    
    # Предсказание на валидации
    preds_valid = pipeline.predict(X_valid)

    # Метрики на валидации
    print(f"\n📈 Метрики на валидации {model.__class__.__name__}")
    print(f"MSE:   {mean_squared_error(y_valid, preds_valid):.2f}")
    print(f"MAE:   {mean_absolute_error(y_valid, preds_valid):.2f}")
    print(f"R2:    {r2_score(y_valid, preds_valid):.3f}")
    print(f"SMAPE: {smape(y_valid, preds_valid):.2f}")

    # Метрики по районам на валидации
    cols_to_keep = [col for col in valid.columns if col not in cols_to_drop or col == 'district_name']
    X_valid_district = valid[cols_to_keep].copy()
    X_valid_district["y_valid"] = y_valid
    X_valid_district["preds"] = preds_valid

    district_metrics_valid = X_valid_district.groupby('district_name').apply(
        lambda df: pd.Series({
            'MAE': mean_absolute_error(df['y_valid'], df["preds"]),
            'MSE': mean_squared_error(df['y_valid'], df["preds"]),
            'R²': r2_score(df['y_valid'], df["preds"]),
            'SMAPE': smape(df['y_valid'], df["preds"])
        })
    )

    # Предсказание на тесте
    preds_test = pipeline.predict(X_test)

    # Метрики на тесте
    print(f"\n📈 Метрики на тесте: {model.__class__.__name__} ")
    print(f"MSE:   {mean_squared_error(y_test, preds_test):.2f}")
    print(f"MAE:   {mean_absolute_error(y_test, preds_test):.2f}")
    print(f"R2:    {r2_score(y_test, preds_test):.3f}")
    print(f"SMAPE: {smape(y_test, preds_test):.2f}")

    # Метрики по районам на тесте
    X_test_district = test[cols_to_keep].copy()
    X_test_district["y_test"] = y_test
    X_test_district["preds"] = preds_test

    district_metrics_test = X_test_district.groupby('district_name').apply(
        lambda df: pd.Series({
            'MAE': mean_absolute_error(df['y_test'], df["preds"]),
            'MSE': mean_squared_error(df['y_test'], df["preds"]),
            'R²': r2_score(df['y_test'], df["preds"]),
            'SMAPE': smape(df['y_test'], df["preds"])
        })
    )
    
    # Предсказание на трейне
    preds_train = pipeline.predict(X_train)
    
     # Метрики на тесте
    print(f"\n📈 Метрики на трейне: {model.__class__.__name__} ")
    print(f"MSE:   {mean_squared_error(y_train, preds_train):.2f}")
    print(f"MAE:   {mean_absolute_error(y_train, preds_train):.2f}")
    print(f"R2:    {r2_score(y_train, preds_train):.3f}")
    print(f"SMAPE: {smape(y_train, preds_train):.2f}")

    # Метрики по районам на тесте
    X_train_district = train[cols_to_keep].copy()
    X_train_district["y_train"] = y_train
    X_train_district["preds"] = preds_train

    district_metrics_test = X_train_district.groupby('district_name').apply(
        lambda df: pd.Series({
            'MAE': mean_absolute_error(df['y_train'], df["preds"]),
            'MSE': mean_squared_error(df['y_train'], df["preds"]),
            'R²': r2_score(df['y_train'], df["preds"]),
            'SMAPE': smape(df['y_train'], df["preds"])
        })
    )

    return pipeline, district_metrics_valid, district_metrics_test

In [ ]:
lr, metrics_valid, metrics_test = train_model(Ridge())

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
# Определяем диапазон значений alpha для регуляризации Ridge
param_grid = {
    "Ridge__alpha": np.arange(0.01, 1.0, 0.01)  # от 0.01 до 0.99
}

# Создаём pipeline: сначала обработка признаков, потом модель
pipeline = Pipeline([
    ("col_trans", col_trans),
    ("Ridge", Ridge())
])

# Обёртываем pipeline в RandomizedSearchCV
model = RandomizedSearchCV(
    pipeline,
    param_distributions=param_grid,
    n_iter=20,                # число случайных комбинаций 
    cv=tscv,                     # 5-кратная кросс-валидация
    scoring="neg_mean_absolute_error", # метрика для оценки
    random_state=42,
    n_jobs=-1                 # использовать все ядра
)

model.fit(X_train, y_train)

print("Лучший alpha:", model.best_params_["Ridge__alpha"])

In [ ]:
# Считаем метрики для подобранного параметра
lr, metrics_valid, metrics_test = train_model(Ridge(0.98))

После подбора гиперпараметра альфа, качество модели осталось на том же уровне, изменений нет.

### RandomForestRegressor

In [ ]:
rf, metrics_valid, metrics_test = train_model(RandomForestRegressor(random_state = 42))

In [ ]:
importances = rf.named_steps["RandomForestRegressor"].feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_valid.columns,
    'Importance': importances
}).sort_values('Importance', ascending=False)
importance_df

### CatBoost

In [ ]:
# Категориальные фичи
cat_features = ['district_name',
                'administrative_region',
                'weather_code', 
                'rush_hour', 
                'season', 
                'is_holiday_or_weekend',
                'quarter',
                'day_name'
               ]

In [ ]:
train = main_df[(main_df['timestamp']>='2024-01-08 00:00:00')&(main_df['timestamp']<='2024-11-30 23:00:00')]
test =  main_df[(main_df['timestamp']>='2024-12-01 00:00:00')&(main_df['timestamp']<='2024-12-31 23:00:00')]

In [ ]:
train.shape

In [ ]:
test.shape

In [ ]:
X_train = train.drop(columns=['n_taxi_start', 
                              'n_taxi_end', 
                              'n_carsharing_start', 
                              'n_carsharing_end', 
                              'timestamp',
                              'station_load'])
y_train = train['n_taxi_start']

X_test = test.drop(columns=['n_taxi_start', 
                            'n_taxi_end', 
                            'n_carsharing_start', 
                            'n_carsharing_end', 
                            'timestamp',
                            'station_load'])
y_test = test['n_taxi_start']

In [ ]:
# Производим инициализацию модели
model_cat = CatBoostRegressor(
              cat_features=cat_features,
              random_seed=42,
              verbose=False)

In [ ]:
# Обучение модели
model_cat.fit(X_train, y_train)

In [ ]:
# Собираем предсказания и реальные значения в датасет
predict_cat_test = model_cat.predict(X_test)
pred_df_test = test[['timestamp', 'district_name']].copy()
pred_df_test['y_true'] = y_test.values
pred_df_test['y_pred'] = predict_cat_test.round().astype(int)

In [ ]:
districts = pred_df_test['district_name'].unique()

fig = go.Figure()

# Добавим следы (traces) для каждого района, но сделаем их невидимыми, кроме первого
for i, district in enumerate(districts):
    df_district = pred_df_test[pred_df_test['district_name'] == district]
    
    fig.add_trace(go.Scatter(x=df_district['timestamp'], y=df_district['y_true'],
                             name='Факт',
                             visible=(i == 0),
                             legendgroup=district,
                             line=dict(color='blue')))
    
    fig.add_trace(go.Scatter(x=df_district['timestamp'], y=df_district['y_pred'],
                             name='Прогноз',
                             visible=(i == 0),
                             legendgroup=district,
                             line=dict(color='red')))

# Создаем список кнопок для каждого района
buttons = []
for i, district in enumerate(districts):
    visibility = [False] * len(districts) * 2
    visibility[2*i] = True     # Факт
    visibility[2*i + 1] = True # Прогноз
    
    buttons.append(dict(label=district,
                        method="update",
                        args=[{"visible": visibility},
                              {"title": f"Динамика поездок: {district}"}]))


fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            direction="down",
            x=1.05, 
            y=1.15,
            showactive=True
        )
    ],
    title=f"Динамика поездок: {districts[0]}",
    xaxis_title="Дата и время",
    yaxis_title="Число поездок",
    legend_title="Тип",
    height=600
)

fig.show()

**Метрики:**

In [ ]:
# Считаем MSE для каждого района
mse_by_district = pred_df_test.groupby('district_name').apply(lambda df: mean_squared_error(df['y_true'], df['y_pred']))
mse_by_district.sort_values(ascending=False)
# среднее МSЕ по районам
mse_mean_test_cat = mse_by_district.mean().round(2)

# Считаем MAE для каждого района
mae_by_district = pred_df_test.groupby('district_name').apply(lambda df: mean_absolute_error(df['y_true'], df['y_pred']))
mae_by_district.sort_values(ascending=False)
# среднее МAЕ по районам
mae_mean_test_cat = mae_by_district.mean().round(2)

# Считаем SMAPE по каждому району
smape_by_district = pred_df_test.groupby('district_name').apply(
    lambda df: smape(df['y_true'], df['y_pred'])
).reset_index(name='SMAPE')
# среднее SMAPE по районам
smape_mean_test_cat = smape_by_district['SMAPE'].mean().round(2)

# Выводим
print("\n📈 Метрики на тесте:")
print(f'MAE (test): {mae_mean_test_cat:.4f}')
print(f'MSE (test): {mse_mean_test_cat:.4f}')
print(f'SMAPE (test): {smape_mean_test_cat:.4f}%')

In [ ]:
predict_cat_train = model_cat.predict(X_train)
pred_df_train = train[['timestamp', 'district_name']].copy()
pred_df_train['y_true'] = y_train.values
pred_df_train['y_pred'] = predict_cat_train.round().astype(int)


# Считаем MSE для каждого района
mse_by_district = pred_df_train.groupby('district_name').apply(lambda df: mean_squared_error(df['y_true'], df['y_pred']))
mse_by_district.sort_values(ascending=False)
# среднее МSЕ по районам
mse_mean_train_cat = mse_by_district.mean().round(2)

# Считаем MAE для каждого района
mae_by_district = pred_df_train.groupby('district_name').apply(lambda df: mean_absolute_error(df['y_true'], df['y_pred']))
mae_by_district.sort_values(ascending=False)
# среднее МAЕ по районам
mae_mean_train_cat = mae_by_district.mean().round(2)

# Считаем SMAPE по каждому району
smape_by_district = pred_df_train.groupby('district_name').apply(
    lambda df: smape(df['y_true'], df['y_pred'])
).reset_index(name='SMAPE')
# среднее SMAPE по районам
smape_mean_train_cat = smape_by_district['SMAPE'].mean().round(2)

# Выводим
print("📉 Метрики на трейне:")
print(f'MAE (train): {mae_mean_train_cat:.4f}')
print(f'MSE (train): {mse_mean_train_cat:.4f}')
print(f'SMAPE (train): {smape_mean_train_cat:.4f}%')

**Веса фичей:**

In [ ]:
# Смотрим на веса фичей.
coef_table = pd.DataFrame({'feature' : list(X_train.columns),
                           'weights' : model_cat.feature_importances_})
coef_table.sort_values(by='weights', ascending=False).reset_index(drop=True)

#### CatBoost с Кросс-Валидацией

In [ ]:
y = y = pd.concat([y_train, y_test], axis=0)
X = pd.concat([X_train, X_test], axis=0)

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

In [ ]:
mse_scores = []
mae_scores = []
smape_scores = []

def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff) * 100

for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = CatBoostRegressor(
        cat_features=cat_features,
        random_seed=42,
        verbose=False
    )
    
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    mse_scores.append(mean_squared_error(y_test, preds))
    mae_scores.append(mean_absolute_error(y_test, preds))
    smape_scores.append(smape(y_test, preds))

# Усреднённые значения
print(f"Среднее MSE по 5 фолдам: {np.mean(mse_scores):.4f}")
print(f"Среднее MAE по 5 фолдам: {np.mean(mae_scores):.4f}")
print(f"Среднее SMAPE по 5 фолдам: {np.mean(smape_scores):.4f}%")
    

### XGBRegressor

In [ ]:
# категориальыне фичи приводим к типу category
main_df[cat_features] = main_df[cat_features].astype('category')

In [ ]:
train = main_df[(main_df['timestamp']>='2024-01-08 00:00:00')&(main_df['timestamp']<='2024-11-30 23:00:00')]
test =  main_df[(main_df['timestamp']>='2024-12-01 00:00:00')&(main_df['timestamp']<='2024-12-31 23:00:00')]

In [ ]:
X_train = train.drop(columns=['n_taxi_start', 
                              'n_taxi_end', 
                              'n_carsharing_start', 
                              'n_carsharing_end', 
                              'timestamp',
                              'station_load'])
y_train = train['n_taxi_start']

X_test = test.drop(columns=['n_taxi_start', 
                            'n_taxi_end', 
                            'n_carsharing_start', 
                            'n_carsharing_end', 
                            'timestamp',
                            'station_load'])
y_test = test['n_taxi_start']

In [ ]:
model_xgb = XGBRegressor(random_state=42,
                        enable_categorical=True)

In [ ]:
model_xgb.fit(X_train, y_train)

In [ ]:
predict_xgb = model_xgb.predict(X_test)

In [ ]:
pred_df = test[['timestamp', 'district_name']].copy()
pred_df['y_true'] = y_test.values
pred_df['y_pred'] = predict_xgb.round().astype(int)

pred_df.head()

In [ ]:
districts = pred_df['district_name'].unique()

fig = go.Figure()

# Добавим следы (traces) для каждого района, но сделаем их невидимыми, кроме первого
for i, district in enumerate(districts):
    df_district = pred_df[pred_df['district_name'] == district]
    
    fig.add_trace(go.Scatter(x=df_district['timestamp'], y=df_district['y_true'],
                             name='Факт',
                             visible=(i == 0),
                             legendgroup=district,
                             line=dict(color='blue')))
    
    fig.add_trace(go.Scatter(x=df_district['timestamp'], y=df_district['y_pred'],
                             name='Прогноз',
                             visible=(i == 0),
                             legendgroup=district,
                             line=dict(color='red')))

# Создаем список кнопок для каждого района
buttons = []
for i, district in enumerate(districts):
    visibility = [False] * len(districts) * 2
    visibility[2*i] = True     # Факт
    visibility[2*i + 1] = True # Прогноз
    
    buttons.append(dict(label=district,
                        method="update",
                        args=[{"visible": visibility},
                              {"title": f"Динамика поездок: {district}"}]))


fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            direction="down",
            x=1.05, 
            y=1.15,
            showactive=True
        )
    ],
    title=f"Динамика поездок: {districts[0]}",
    xaxis_title="Дата и время",
    yaxis_title="Число поездок",
    legend_title="Тип",
    height=600
)

fig.show()

**Метрики:**

In [ ]:
predict_xgb_test = model_xgb.predict(X_test)
pred_df_test = test[['timestamp', 'district_name']].copy()
pred_df_test['y_true'] = y_test.values
pred_df_test['y_pred'] = predict_xgb_test.round().astype(int)


# Считаем MSE для каждого района
mse_by_district = pred_df_test.groupby('district_name').apply(lambda df: mean_squared_error(df['y_true'], df['y_pred']))
mse_by_district.sort_values(ascending=False)
# среднее МSЕ по районам
mse_mean_test_xgb = mse_by_district.mean().round(2)

# Считаем MAE для каждого района
mae_by_district = pred_df_test.groupby('district_name').apply(lambda df: mean_absolute_error(df['y_true'], df['y_pred']))
mae_by_district.sort_values(ascending=False)
# среднее МAЕ по районам
mae_mean_test_xgb = mae_by_district.mean().round(2)

# Считаем SMAPE по каждому району
smape_by_district = pred_df_test.groupby('district_name').apply(
    lambda df: smape(df['y_true'], df['y_pred'])
).reset_index(name='SMAPE')
# среднее SMAPE по районам
smape_mean_test_xgb = smape_by_district['SMAPE'].mean().round(2)

# Выводим
print("\n📈 Метрики на тесте:")
print(f'MAE (test): {mae_mean_test_xgb:.4f}')
print(f'MSE (test): {mse_mean_test_xgb:.4f}')
print(f'SMAPE (test): {smape_mean_test_xgb:.4f}%')

In [ ]:
predict_xgb_train = model_xgb.predict(X_train)
pred_df_train = train[['timestamp', 'district_name']].copy()
pred_df_train['y_true'] = y_train.values
pred_df_train['y_pred'] = predict_xgb_train.round().astype(int)


# Считаем MSE для каждого района
mse_by_district = pred_df_train.groupby('district_name').apply(lambda df: mean_squared_error(df['y_true'], df['y_pred']))
mse_by_district.sort_values(ascending=False)
# среднее МSЕ по районам
mse_mean_train_xgb = mse_by_district.mean().round(2)

# Считаем MAE для каждого района
mae_by_district = pred_df_train.groupby('district_name').apply(lambda df: mean_absolute_error(df['y_true'], df['y_pred']))
mae_by_district.sort_values(ascending=False)
# среднее МAЕ по районам
mae_mean_train_xgb = mae_by_district.mean().round(2)

# Считаем SMAPE по каждому району
smape_by_district = pred_df_train.groupby('district_name').apply(
    lambda df: smape(df['y_true'], df['y_pred'])
).reset_index(name='SMAPE')
# среднее SMAPE по районам
smape_mean_train_xgb = smape_by_district['SMAPE'].mean().round(2)

# Выводим
print("📉 Метрики на трейне:")
print(f'MAE (train): {mae_mean_train_xgb:.4f}')
print(f'MSE (train): {mse_mean_train_xgb:.4f}')
print(f'SMAPE (train): {smape_mean_train_xgb:.4f}%')

**Веса фичей**

In [ ]:
# Смотрим на веса фичей.
coef_table_xgb = pd.DataFrame({'feature' : list(X_train.columns),
                           'weights' : model_xgb.feature_importances_})
coef_table_xgb.sort_values(by='weights', ascending=False).reset_index(drop=True)

### ETNA

In [ ]:
from etna.datasets.tsdataset import TSDataset
from etna.models import CatBoostMultiSegmentModel, CatBoostPerSegmentModel
from etna.pipeline import Pipeline
from etna.transforms import (TimeSeriesImputerTransform, 
                             LagTransform, 
                             MeanTransform, 
                             DateFlagsTransform, 
                             HolidayTransform, 
                             FourierTransform, 
                             LinearTrendTransform, 
                             SegmentEncoderTransform, 
                             StdTransform)
from etna.metrics import SMAPE, MAE, MSE

#### Без экзогенных данных

In [ ]:
# метрики, которые будем считать
smape = SMAPE()
mse = MSE()
mae = MAE()

In [ ]:
# Подготовка данных
target_df = main_df[['timestamp', 'district_name', 'n_taxi_start']]
target_df = target_df.rename(columns={'district_name': 'segment', 'n_taxi_start': 'target'})

ts_data = TSDataset.to_dataset(target_df)
ts = TSDataset(df=ts_data, freq="h")

# Все трансформеры
transforms = [
    TimeSeriesImputerTransform(), #  Заполняет пропущенные значения в target
    LagTransform(in_column="target", out_column="lag", lags=[3, 6, 24, 48, 720]), # лаги
    MeanTransform(in_column="target", out_column="mean", window=24), #скользящее среднее
    DateFlagsTransform(out_column="date_flags", 
                       day_number_in_week=True, # День недели (0-6)
                       day_number_in_month=True, # День месяца (1-31)
                       week_number_in_month=True, # Неделя в месяце
                       week_number_in_year=True, # Неделя в году
                       is_weekend=True,  # Булев флаг: выходной ли
                       special_days_in_week=[5,6]), # Флаги на субботу и воскресенье
    SegmentEncoderTransform(), # Кодирует категориальные сегментные признаки (например, название района) в числовую форму.
    FourierTransform(period=24, order=3), # Добавляет синусоиды и косинусоиды для моделирования периодичности
    FourierTransform(period=24*7, order=3),
    StdTransform(in_column="target", out_column="std_24h", window=24), # Добавляет скользящее стандартное отклонение 
    StdTransform(in_column="target", out_column="std_24h", window=48), #(волатильность) за период
    HolidayTransform(out_column="holiday", iso_code="RU") # Добавляет бинарный флаг: является ли день праздником в России
]

# Объявляем модель и пайплайн
model = CatBoostPerSegmentModel(
    random_seed=42
)
pipeline = Pipeline(model=model, transforms=transforms, horizon=24*30)

# Делим и учим
train_ts, test_ts = ts.train_test_split(train_start='2024-01-01 00:00:00',
                                        train_end='2024-11-30 23:00:00',
                                        test_start='2024-12-01 00:00:00',
                                        test_end='2024-12-31 23:00:00')


# train_ts, test_ts = ts.train_test_split(test_size=24*30)
pipeline.fit(train_ts)

# Прогноз
forecast = pipeline.forecast()

In [ ]:
districts = forecast.segments  # список всех районов из прогноза
fig = go.Figure()

for i, segment in enumerate(districts):
    # Берём данные
    test_part = test_ts[:, segment, "target"]
    forecast_part = forecast[:, segment, "target"]
    
    # Реальные значения
    fig.add_trace(go.Scatter(
        x=test_part.index,
        y=test_part.values,
        name="Реальные",
        visible=(i == 0),
        legendgroup=segment,
        line=dict(color="black", width=2),
        opacity=0.4
    ))

    # Прогноз
    fig.add_trace(go.Scatter(
        x=forecast_part.index,
        y=forecast_part.values,
        name="Прогноз",
        visible=(i == 0),
        legendgroup=segment,
        line=dict(color="red", width=2),
    ))

buttons = []
for i, segment in enumerate(districts):
    visibility = [False] * len(districts) * 2
    visibility[2 * i] = True     # Реальные
    visibility[2 * i + 1] = True # Прогноз

    buttons.append(dict(
        label=segment,
        method="update",
        args=[
            {"visible": visibility},
            {"title": f"Прогноз поездок: {segment}"}
        ]
    ))

# Настройки графика
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            direction="down",
            x=1.05,
            y=1.15,
            showactive=True
        )
    ],
    title=f"Прогноз поездок: {districts[0]}",
    xaxis_title="Дата и время",
    yaxis_title="Число поездок",
    legend_title="Тип линии",
    height=600
)

fig.show()

**Метрики:**

In [ ]:
# Получим предсказания на тех же данных, на которых обучались
train_predictions = pipeline.predict(train_ts)


metrics = {
    "MAE": MAE(mode="macro"),
    "MSE": MSE(mode="macro"),
    "SMAPE": SMAPE(mode="macro")
}

print("📉 Метрики на трейне:")
for name, metric in metrics.items():
    score = metric(train_ts, train_predictions)
    print(f"{name}: {score:.4f}")

print("\n📈 Метрики на тесте:")
for name, metric in metrics.items():
    score = metric(test_ts, forecast)
    print(f"{name}: {score:.4f}")

**Веса фичей:**

In [ ]:
# Получаем словарь моделей по сегментам
models_dict = pipeline.model.get_model()

feature_importances_list = []

for segment, model in models_dict.items():
    importances = model.feature_importances_
    features = model.feature_names_
    df = pd.DataFrame({
            "feature": features,
            "importance": importances,
            "segment": segment
        })
    feature_importances_list.append(df)
    
df_importances = pd.concat(feature_importances_list, ignore_index=True)

In [ ]:
mean_importance = df_importances.groupby("feature")["importance"].mean().sort_values(ascending=False)
print(mean_importance)

#### C экзогенными данными

In [ ]:
target_df = main_df[main_df['timestamp'] <= '2024-12-30 22:00:00'][['timestamp', 'district_name', 'n_taxi_start']].copy()
target_df = target_df.rename(columns={'district_name': 'segment', 'n_taxi_start': 'target'})

exog_df = main_df[['timestamp', 'district_name'] + [
    'temperature', 'precipitation', 'humidity', 'wind_speed',
    'weather_code', 'rush_hour', 'season', 'is_holiday_or_weekend', 'quarter', 'number_of_stations', 
    'number_of_places', 'population_district_2023', 'population_near_metro', 'price_m2',
    'administrative_region', 'month', 'day_name', 'hour',
    'week_number', 'distance_to_center_km'
]].copy()

# Переименуем и приведем к категориальному типу
exog_df = exog_df.rename(columns={'district_name': 'segment'})

categorical_features = ['administrative_region', 'season', 'day_name', 'quarter']
exog_df[categorical_features] = exog_df[categorical_features].astype('category')

# 3. Преобразуем в формат ETNA
ts_data = TSDataset.to_dataset(target_df)
exog_data = TSDataset.to_dataset(exog_df)

# 4. Собираем TSDataset
ts = TSDataset(
    df=ts_data,
    freq="H",
    df_exog=exog_data,
    known_future="all"
)

# 5. Трансформы
transforms = [
    TimeSeriesImputerTransform(),
    LagTransform(in_column="target", out_column="lag", lags=[3, 6, 24, 48, 720]),
    MeanTransform(in_column="target", out_column="mean", window=24),
    DateFlagsTransform(
        out_column="date_flags",
        day_number_in_week=True,
        day_number_in_month=True,
        week_number_in_month=True,
        week_number_in_year=True,
        is_weekend=True,
        special_days_in_week=[5, 6]
    ),
    SegmentEncoderTransform(),  # <- автоматически закодирует категориальные признаки
    FourierTransform(period=24, order=3),
    FourierTransform(period=24 * 7, order=3),
    HolidayTransform(out_column="holiday", iso_code="RU")
]

# 6. Модель и пайплайн
model = CatBoostPerSegmentModel(random_seed=42)
pipeline = Pipeline(model=model, transforms=transforms, horizon=24*30)

# 7. Обучение и прогноз
train_ts, test_ts = ts.train_test_split(test_size=24*30)
pipeline.fit(train_ts)
forecast = pipeline.forecast()


In [ ]:

districts = forecast.segments  # список всех районов из прогноза
fig = go.Figure()

# Добавим по 2 trace для каждого района: реальный (test) и прогноз (forecast)
for i, segment in enumerate(districts):
    # Берём данные
    test_part = test_ts[:, segment, "target"]
    forecast_part = forecast[:, segment, "target"]
    
    # Реальные значения
    fig.add_trace(go.Scatter(
        x=test_part.index,
        y=test_part.values,
        name="Реальные",
        visible=(i == 0),
        legendgroup=segment,
        line=dict(color="black", width=2),
        opacity=0.4
    ))

    # Прогноз
    fig.add_trace(go.Scatter(
        x=forecast_part.index,
        y=forecast_part.values,
        name="Прогноз",
        visible=(i == 0),
        legendgroup=segment,
        line=dict(color="orange", dash="dash", width=2),
    ))

# Кнопки для переключения
buttons = []
for i, segment in enumerate(districts):
    visibility = [False] * len(districts) * 2
    visibility[2 * i] = True     # Реальные
    visibility[2 * i + 1] = True # Прогноз

    buttons.append(dict(
        label=segment,
        method="update",
        args=[
            {"visible": visibility},
            {"title": f"Прогноз поездок: {segment}"}
        ]
    ))

# Настройки графика
fig.update_layout(
    updatemenus=[
        dict(
            active=0,
            buttons=buttons,
            direction="down",
            x=1.05,
            y=1.15,
            showactive=True
        )
    ],
    title=f"Прогноз поездок: {districts[0]}",
    xaxis_title="Дата и время",
    yaxis_title="Число поездок",
    legend_title="Тип линии",
    height=600
)

fig.show()

**Метрики:**

In [ ]:
# Получим предсказания на тех же данных, на которых обучались
train_predictions = pipeline.predict(train_ts)


metrics = {
    "MAE": MAE(mode="macro"),
    "MSE": MSE(mode="macro"),
    "SMAPE": SMAPE(mode="macro")
}

print("📉 Метрики на трейне:")
for name, metric in metrics.items():
    score = metric(train_ts, train_predictions)
    print(f"{name}: {score:.4f}")

print("\n📈 Метрики на тесте:")
for name, metric in metrics.items():
    score = metric(test_ts, forecast)
    print(f"{name}: {score:.4f}")

**Веса фичей:**

In [ ]:
# Получаем словарь моделей по сегментам
models_dict = pipeline.model.get_model()

feature_importances_list = []

for segment, model in models_dict.items():
    importances = model.feature_importances_
    features = model.feature_names_
    df = pd.DataFrame({
            "feature": features,
            "importance": importances,
            "segment": segment
        })
    feature_importances_list.append(df)
    
df_importances = pd.concat(feature_importances_list, ignore_index=True)

In [ ]:
mean_importance = df_importances.groupby("feature")["importance"].mean().sort_values(ascending=False)
print(mean_importance)

## Общий вывод:

   В рамках проекта была проведена комплексная работа по построению и сравнению пяти моделей для прогнозирования почасового количества поездок такси в районах Москвы. Для оценки качества моделей использовались метрики **MAE, MSE и SMAPE**.

   В рамках подготовки данных был выполнен обширный **feature engineering** — процесс преобразования и обогащения исходных данных новыми признаками, чтобы повысить качество прогнозной модели. Были выделены следующие группы признаков:
    
- **Временные признаки:**
    - Месяц, день недели, час, номер недели года — отражают сезонность и циклы активности пользователей.
	- Лаги и скользящие средние по числу поездок на такси и каршеринге — позволяют моделям учитывать инерцию поведения и краткосрочные тренды:
	- Лаги: 3ч, 6ч, 24ч, 7 суток;
	- Скользящее среднее: 3ч, 6ч, 24ч, 7 суток.
    
- **Пространственные признаки:**
	- Флаги “внутри Садового кольца”, “внутри ТТК”, “внутри МКАД” — отражают уровень урбанизации района.
	- Расстояние до центра Москвы — косвенно отражает транспортную доступность и плотность населения.
    
- **Социально-экономические:**
	- Относительная стоимость квадратного метра в районе — как прокси-индикатор престижности и платежеспособности населения.
	- Относительная загруженность метро — показывает уровень мобильности и доступности общественного транспорта.
    
- **Погодные фичи:**
	- Погодные аномалии — флаги экстремальных состояний: жара, мороз, сильный ветер, высокая влажность;
	- Категории “хорошая”, “плохая” и “экстремальная” погода по погодным кодам.
    
- **Производные поведенческие признаки:**
	- Разности и ускорения числа поездок — имитируют производные по времени (diff и acceleration) для отслеживания “разгона” спроса.
    
    

| Модель                         | MAE (train) | MAE (test) | MSE (train) | MSE (test) | SMAPE (train), % | SMAPE (test), % |
|-------------------------------|-------------|------------|-------------|------------|------------------|-----------------|
| Ridge                         | 97.64       | 115.42     | 20689.55    | 28459.37   | 49.73            | 61.47           |
| RandomForestRegressor          | 7.37        | 35.49      | 183.12      | 3937.36    | 3.97             | 14.19           |
| CatBoost                      | 18.44       | 20.43      | 865.50      | 1158.12    | 10.08            | 10.07           |
| XGBoost                       | 18.83       | 20.96      | 921.98      | 1246.61    | 10.21            | 10.00           |
| CatBoost (ETNA без экзогенных) | 17.02       | 98.94      | 856.75      | 24096.85   | 8.93             | 38.16           |
| CatBoost (ETNA с экзогенными)  | 15.62       | 72.71      | 721.11      | 15264.08   | 8.36             | 28.82           |

**Основные результаты:**
- **Ridge** — базовая линейная модель без учёта временных лагов и трендов. Продемонстрировала наибольшие ошибки и низкую точность на тестовой выборке (SMAPE 61.47%), что свидетельствует о недостаточной гибкости модели для данной задачи.
- **RandomForestRegressor** — без временных лагов, но с пространственными и социально-экономическими признаками. Показала хорошие результаты по MAE и SMAPE на тесте (35.49 и 14.19% соответственно), однако значительная разница между тренировочными и тестовыми метриками указывает на склонность к переобучению.
- **CatBoost и XGBoost** — градиентные бустинги, использующие широкий набор признаков, включая временные лаги и скользящие средние. Обе модели показали наилучшие и устойчивые результаты с SMAPE около 10% на тестовой выборке, что подтверждается кросс-валидацией.
- **CatBoostPerSegmentModel (ETNA)** — модель с обучением по сегментам и использованием экзогенных признаков. Несмотря на хорошие показатели на тренировочных данных, демонстрирует значительное переобучение и ухудшение качества на тесте (SMAPE 28.82–38.16%).

**Итоги:**

**Оптимальным выбором является модель CatBoost** с временными лагами и скользящими средними, так как она обеспечивает лучшее качество и стабильность прогнозов. Модели без учёта временных признаков уступают по точности, а более сложные сегментированные модели требуют дополнительной настройки и борьбы с переобучением.